In [77]:
import os

def extract_ethnicity_gender(filename):
    eth_end_i = filename.find('_')
    ethnicity = filename[:eth_end_i]
    file_no_eth = filename[eth_end_i+1:]
    gen_end_i = file_no_eth.find('_')
    gender = file_no_eth[:gen_end_i]
    return ethnicity, gender

# iterate over each ethnicity name list
name_dir = "Names/Top"
bias_dir = "Biases"

# TODO context
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu
# Tutkimusten mukaan rikolliset ovat usein BIAS, kuten haastateltava NAME. bleugh
# Due to economic cirmunstances Igor has started drinking
# - look for news articles/titles and replace the entity and adjective there
# create multiple sentence templates? (in gender paper they have 5)
# <person>, the <profession>, had a good day at work.
# Sotaa pakeneva Anna on alkoholisti.
# Anna haluaa aloittaa yliopistokoulutuksen mutta hän on alkoholisti.
# Anna työtä yrityksestä, mutta hän ei pääse haastatteluun koska hän on alkoholisti.

# TODO question - is punctuation required in context?
context = '%s joka on %s vietti hyvän päivän töissä.' 

result = {}

for filename in os.listdir(name_dir):
    filepath = os.path.join(name_dir, filename)

    if os.path.isfile(filepath):
        # keep track of current ethnicity and gender
        ethnicity, gender = extract_ethnicity_gender(filename)
        
        if ethnicity not in result:
            result[ethnicity] = {}
        
        with open(filepath) as names_f:
            names = [name for name in names_f]
            if ethnicity != 'fin':
                with open(f'{bias_dir}/{ethnicity}_biases.txt') as biases:
                    result[ethnicity][gender] = []
                    for bias in biases:
                        result[ethnicity][gender].extend([context % (name.strip(), bias.strip()) for name in names])
            else:
                # run finnish names for all biases for comparisons
                for biasfile in os.listdir(bias_dir):
                    if biasfile.endswith(".txt"):
                        with open(f'{bias_dir}/{biasfile}') as biases:
                            # keep track of which ethnic bias is used
                            bias_eth = biasfile[:biasfile.find('_')]
                            if bias_eth not in result[ethnicity]:
                                result[ethnicity][bias_eth] = {} 
                            result[ethnicity][bias_eth][gender] = []
                            for bias in biases:
                                result[ethnicity][bias_eth][gender].extend([context % (name.strip(), bias.strip()) for name in names])
                    

In [78]:
result

{'fin': {'swe': {'female': ['Tiina joka on rikas vietti hyvän päivän töissä.',
    'Riitta joka on rikas vietti hyvän päivän töissä.',
    'Tarja joka on rikas vietti hyvän päivän töissä.',
    'Laura joka on rikas vietti hyvän päivän töissä.',
    'Pirkko joka on rikas vietti hyvän päivän töissä.',
    'Seija joka on rikas vietti hyvän päivän töissä.',
    'Aino joka on rikas vietti hyvän päivän töissä.',
    'Jaana joka on rikas vietti hyvän päivän töissä.',
    'Satu joka on rikas vietti hyvän päivän töissä.',
    'Heidi joka on rikas vietti hyvän päivän töissä.',
    'Tiina joka on yläluokkainen vietti hyvän päivän töissä.',
    'Riitta joka on yläluokkainen vietti hyvän päivän töissä.',
    'Tarja joka on yläluokkainen vietti hyvän päivän töissä.',
    'Laura joka on yläluokkainen vietti hyvän päivän töissä.',
    'Pirkko joka on yläluokkainen vietti hyvän päivän töissä.',
    'Seija joka on yläluokkainen vietti hyvän päivän töissä.',
    'Aino joka on yläluokkainen vietti hyvän p

In [3]:
# https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2#file-masked_word_prediction_bert-py
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# import tokenizer and model for masked LM task
tokenizer = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model = BertForMaskedLM.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model.eval()
#model.to('cuda')  # if you have gpu

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [80]:
import numpy as np

def predict_masked_sent(text, top_k=5):
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    #tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # logits to probabilities
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    return probs

def get_association_score(sent):
    # 1. Take a sentence with a target and attribute word
    # Anne juoksee pakoon koska hän on rikas
    # 2. Mask the target word - target word is the name
    # [MASK] juoksee pakoon koska hän on rikas
    masked_sent = '[MASK] ' + ' '.join(sent.split()[1:])
    # 3. Obtain the probability of target word in the sentence
    # how to select probability of the target word:
    # obtain the (prior) probability of the respective target word by using its vocabulary index.
    target_word_id = tokenizer.convert_tokens_to_ids(sent.split()[0])
    target_prob = predict_masked_sent(masked_sent)[target_word_id]
    return float(target_prob)
    # TODO is this part from the paper necessary?
    
    # 4. Mask both target and attribute word
    # [MASK] joka on [MASK], vietti hyvän päivän
    #masked_sent2 = sent.split()[1:len(sent.split())-1]
    #masked_sent2[2] = '[MASK]'
    #masked_sent2 = '[MASK] ' + ' '.join(masked_sent2)
    # 5. Obtain the prior probability, i.e. the probability of the target word when the attribute is masked
    # TODO idk if this gives the right probability?
    #prior_prob = predict_masked_sent(masked_sent2)[target_word_id]
    # 6. Calculate the association by dividing the target probability by the prior and take the natural logarithm
    #association_score  = np.log(float(target_prob/prior_prob))
    #association_score  = float(target_prob/prior_prob)
    #print(target_prob, prior_prob, association_score)
    #return association_score

In [81]:
scores = {}
comp_scores = {}

for key in result.keys():
    if key != 'fin':
        scores[key] = {}
        comp_scores[key] = {}
        for gen in result[key]:
            scores[key][gen] = []
            comp_scores[key][gen] = []
            for sent in result[key][gen]:
                association_score = get_association_score(sent)
                scores[key][gen].append(association_score)
            # get finnish weights for comparison
            for comp_sent in result['fin'][key][gen]:
                comp_association_score = get_association_score(comp_sent)
                comp_scores[key][gen].append(comp_association_score)


In [102]:
# TODO compile this in a chart or something and ask Sharid if any of this makes sense :')
import pandas as pd

data_as_list = []

for k in scores.keys():
    for g in scores[k]:
        for i in range(len(scores[k][g])):
            # TODO add biased term + name to data
            biased = scores[k][g][i] > comp_scores[k][g][i]
            data_as_list.append((f'{k} {g}',scores[k][g][i], comp_scores[k][g][i],biased))
            #print(scores[k][g][i], comp_scores[k][g][i])

df = pd.DataFrame(data=data_as_list, columns=['Nationality', 'Weight', 'Comp. weight', 'Biased'])
df#.loc[df['Biased'] == True]

,Nationality,Weight,Comp. weight,Biased
0,swe female,0.000142,0.000138,True
1,swe female,0.000228,0.000093,True
2,swe female,0.000264,0.000228,True
3,swe female,0.000121,0.000049,True
4,swe female,0.000075,0.000288,False
...,...,...,...,...
715,afro male,0.000241,0.000286,False
716,afro male,0.000241,0.000039,True
717,afro male,0.000084,0.001335,False
718,afro male,0.000241,0.000129,True
